# コード開発環境と実行環境（Studio Notebooks と Training Jobs) で同じ環境(コンテナイメージ)を使用する
1_ready.ipynb で作成した環境を Studio Notebooks と Training Jobs で利用する

## 使用するカーネルの設定

右上の部分をクリックして、 `1_ready.ipynb` で作成した、カーネルを選択する
![change_kernel](./img/change_kernel.png)
![select_kernel](./img/select_kernel.png)

選択したカーネルでコードを開発する前に、Dockerfile や requirements.txt を確認してコンテナを確認する

In [ ]:
!cat /root/Dockerfile

In [ ]:
!cat /root/requirements.txt

コード開発前にトレーニングジョブ実行に必要な定数を設定する

In [ ]:
import yaml
import sagemaker
import boto3
region = boto3.session.Session().region_name
account = boto3.client('sts').get_caller_identity().get('Account')


# 定数ファイル読み込み
with open('constant_config.yml','rt') as f:
    constant_config = yaml.safe_load(f)

REPOSITORY_NAME = constant_config['REPOSITORY_NAME']
IMAGE_NAME = constant_config['IMAGE_NAME']
role = sagemaker.get_execution_role()
image_uri = f'{account}.dkr.ecr.{region}.amazonaws.com/{REPOSITORY_NAME}:{IMAGE_NAME}'
print(f'region: {region}')
print(f'account : {account}')
print(f'role : {role}')
print(f'REPOSITORY_NAME : {REPOSITORY_NAME}')
print(f'IMAGE_NAME : {IMAGE_NAME}')
print(f'image_uri : {image_uri}')

コードを開発する。今回は wx+b をするコードを開発したこととする

In [ ]:
# 出来上がったコード
import numpy as np

np.random.seed(seed=1234)
w = np.random.rand(8).reshape(4,2)
x  = np.random.rand(4)
b =  np.random.rand(2)

print(np.dot(x,w)+b)

トレーニングジョブで動かすために .py ファイルに書き出す

In [ ]:
%%writefile train.py

import numpy as np

np.random.seed(seed=1234)
w = np.random.rand(8).reshape(4,2)
x  = np.random.rand(4)
b =  np.random.rand(2)

print(np.dot(x,w)+b)

.py ファイルを用いてトレーニングジョブを動かす。開発したコードと同じ結果が得られることを確認する  

In [ ]:
estimator = sagemaker.estimator.Estimator(
    image_uri = image_uri,
    role = sagemaker.get_execution_role(),
    entry_point='train.py',
    instance_type = 'ml.m5.xlarge',
    instance_count=1
)
estimator.fit()

このように、開発環境と実行環境を統一することで、環境差異に悩まされずに開発をすすめることができる